<a href="https://colab.research.google.com/github/JeffRubens/GitHub/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)
# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. Para saber quando será a próxima edição, o ideal é:

*   **Acompanhar as redes sociais e o site da Alura:** Eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar a página de cursos de IA da Alura:** Veja se a imersão já está listada com uma data prevista.
*   **Entrar em contato com a Alura:** Você pode perguntar diretamente para a equipe deles sobre a previsão da próxima Imersão.

In [7]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={'tools': [{'google_search': {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias **12 e 16 de maio de 2025.** As inscrições estão abertas e podem ser feitas no site da Alura. O curso é gratuito e online, e ao final você recebe um certificado Alura + Google.


In [8]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA Alura Google Gemini', 'Alura Imersão IA Google Gemini datas']
Páginas utilizadas na resposta: eucapacito.com.br, alura.com.br, educacao.mg.gov.br



In [9]:
# Instalar Framework ADK de agentes do Google ################################################
#!pip install google-generative-ai-agent-framework
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes a data de hoje.
        """,
        description='Agente que busca informação no Google',
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [17]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais
        recentes e relevantes buscador, você deve:
        Usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais
        relevantes que pederíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search)
        para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e e retornar esse
        tema, seus pontos mais relevantes e um plano com os assuntos a serem abordados no post que será criado posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [31]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [32]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [33]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Por favor, digite um tópico válido.")
else:
  print("Maravilha! Vamos criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico,data_de_hoje)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------------")

  plano_de_post = agente_planejador(topico,lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planerjador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico,plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------------")

  post_final = agente_revisor(topico,rascunho_de_post)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: renda extra
Maravilha! Vamos criar o post sobre novidades em {topico}

--- Resultado do Agente 1 (Buscador) ---



> Para encontrar as formas mais relevantes e recentes de renda extra, farei algumas buscas no Google. Procurarei por notícias, artigos e discussões sobre novas oportunidades e tendências no cenário de renda extra.
> 
> 
> Com base nas informações mais recentes e relevantes, aqui estão algumas das principais tendências e oportunidades para gerar renda extra em maio de 2025:
> 
> 1.  **Venda de Artesanato:** Se você tem habilidades manuais, criar e vender artesanato online é uma excelente opção. A Nuvemshop destaca que essa pode ser uma ideia lucrativa, especialmente se você já tem um hobby como crochê, fabricação de sabonetes artesanais ou criação de acessórios. Plataformas como Elo7 são ótimas para vender seus produtos, e você também pode criar sua própria loja virtual.
> 2.  **Marketing de Afiliados:** Promover produtos ou serviços de outras empresas e receber comissões por cada venda realizada através de seus links personalizados continua sendo uma forma eficiente de renda extra online. A Hostinger menciona que os ganhos podem variar de 5% a 50% das vendas, dependendo do programa de afiliados.
> 3.  **Serviços Freelance:** Oferecer suas habilidades em áreas como design gráfico, redação, programação ou tradução em plataformas como Workana, Fiverr e 99Freelas é uma maneira flexível de trabalhar de casa e definir seus próprios horários. RealTime1 e Melhor Investimento apontam que essas plataformas oferecem uma vasta gama de oportunidades para quem possui habilidades específicas.
> 4.  **Criação e Venda de Produtos Digitais:** A criação de produtos digitais como e-books, templates e cursos online oferece alto potencial de renda passiva. A Shopify Brasil destaca que o trabalho é feito uma vez e o produto pode ser vendido repetidamente, com margens de lucro elevadas, sem necessidade de armazenamento ou produção contínua.
> 5.  **E-commerce e Revenda de Produtos:** Montar uma loja virtual para vender produtos de nichos específicos como cosméticos, decoração, confecção ou livros é uma ótima maneira de gerar renda extra online. A Nuvemshop e RealTime1 mencionam que a revenda de produtos no atacado também continua popular, com a possibilidade de vender em marketplaces como Mercado Livre e OLX.
> 
> Além dessas opções, outras ideias relevantes incluem:
> 
> *   **Aluguel de Espaços ou Produtos:** Alugar quartos, casas de temporada, ferramentas ou equipamentos através de plataformas como Airbnb e OLX.
> *   **Serviços de Transporte e Logística:** Atuar como motorista de aplicativos como Uber e 99 ou como entregador de comida através de plataformas como iFood e Rappi.
> *   **Consultoria Online:** Oferecer consultoria especializada em áreas como negócios, marketing, finanças ou desenvolvimento pessoal.
> *   **Venda de Itens Usados:** Comercializar produtos usados em bom estado através de plataformas como OLX, Enjoei, Mercado Livre e Facebook Marketplace.
> *   **Pet Sitting e Passeios com Animais:** Oferecer serviços de cuidado e passeios com animais, uma opção altamente procurada por quem ama pets.
> *   **Gestão de Redes Sociais:** Oferecer serviços de criação de conteúdo, agendamento de posts e interação com seguidores para pequenas empresas e autônomos.
> 
> Essas são algumas das maneiras mais promissoras de gerar renda extra em 2025, aproveitando as tendências do mercado e as novas plataformas digitais. Avalie suas habilidades, recursos e interesses para identificar as opções mais adequadas ao seu perfil e começar a aumentar sua renda.
> 


--------------------------------------------------------------------

--- Resultado do Agente 2 (Planerjador) ---



> Okay, com base nas informações fornecidas, vou criar um plano de conteúdo para posts sobre renda extra.
> 
> **Primeiramente, farei algumas pesquisas adicionais para aprofundar meu conhecimento sobre cada tema:**
> 
> 
> Com base nas minhas pesquisas e nas informações que você me forneceu, o tema mais relevante para um post sobre renda extra em maio de 2025 é:
> 
> **Serviços Freelance: A Flexibilidade que Gera Renda Extra**
> 
> **Pontos Relevantes:**
> 
> *   **Alta Demanda:** Empresas e indivíduos estão constantemente buscando freelancers para diversas tarefas, desde design gráfico até redação e programação.
> *   **Flexibilidade:** Permite trabalhar de casa, definir horários e escolher projetos que se encaixem em suas habilidades e interesses.
> *   **Variedade de Habilidades:** Há espaço para praticamente qualquer habilidade que possa ser oferecida online.
> *   **Plataformas:** Workana, Fiverr, 99Freelas, Upwork, Toptal e outras oferecem um mercado vasto de oportunidades.
> *   **Tendências:** Marketing digital freelance, consultoria online e criação de conteúdo estão em alta.
> *   **Importância de um bom perfil:** É essencial construir um perfil atraente e detalhado nas plataformas, demonstrando suas habilidades e experiências.
> *   **Construção de reputação:** Começar com projetos menores para construir uma reputação sólida e receber avaliações positivas.
> *   **Gerenciamento de tempo e finanças:** É importante estabelecer uma rotina de trabalho e reservar parte dos ganhos para possíveis impostos e períodos de menor demanda.
> *   **Marketing pessoal:** Promover seus serviços em redes sociais e criar um portfólio online para atrair mais clientes.
> *   **Especialização:** Focar em um nicho específico para se destacar como especialista e atrair clientes mais qualificados.
> 
> **Plano para o Post:**
> 
> 1.  **Título:** "Serviços Freelance: A Flexibilidade que Gera Renda Extra em 2025"
> 2.  **Introdução:**
>     *   Destaque a crescente busca por renda extra e a popularidade dos serviços freelance.
>     *   Mencione a flexibilidade e variedade de oportunidades que o trabalho freelance oferece.
> 3.  **O que são Serviços Freelance?**
>     *   Explique o conceito de trabalho freelance e como ele se encaixa no cenário atual.
>     *   Destaque a diferença entre ser freelancer e ter um emprego tradicional.
> 4.  **Habilidades em Demanda:**
>     *   Liste as habilidades mais procuradas no mercado freelance em 2025:
>         *   Design Gráfico
>         *   Redação e Edição de Conteúdo
>         *   Programação e Desenvolvimento Web
>         *   Marketing Digital (SEO, Redes Sociais, etc.)
>         *   Tradução
>         *   Consultoria Online
>         *   Assistência Virtual
>     *   Incentive os leitores a identificarem suas próprias habilidades e como elas podem ser oferecidas como serviços.
> 5.  **Plataformas Freelance:**
>     *   Apresente as principais plataformas para encontrar trabalho freelance:
>         *   Workana
>         *   Fiverr
>         *   99Freelas
>         *   Upwork
>     *   Destaque as características e vantagens de cada plataforma.
>     *   Forneça dicas sobre como criar um perfil atraente e eficaz em cada plataforma.
> 6.  **Dicas para Iniciantes:**
>     *   Construir um portfólio online com seus melhores trabalhos.
>     *   Começar com projetos menores para ganhar experiência e avaliações positivas.
>     *   Definir preços competitivos e aumentar gradualmente conforme sua reputação cresce.
>     *   Gerenciar seu tempo e finanças de forma eficiente.
>     *   Comunicar-se de forma clara e profissional com os clientes.
> 7.  **Tendências em Serviços Freelance:**
>     *   Marketing digital e criação de conteúdo para redes sociais
>     *   Consultoria online em diversas áreas (negócios, finanças, etc.)
>     *   Desenvolvimento de aplicativos e soluções web personalizadas
>     *   Serviços de tutoria e ensino online
> 8.  **Marketing Pessoal para Freelancers:**
>     *   Promover seus serviços em redes sociais (LinkedIn, Instagram, etc.).
>     *   Criar um site ou blog para exibir seu trabalho e atrair clientes.
>     *   Participar de grupos e fóruns online relacionados à sua área de atuação.
>     *   Pedir recomendações de clientes satisfeitos.
> 9.  **Histórias de Sucesso:**
>     *   Compartilhe exemplos de freelancers que alcançaram sucesso e renda extra significativa através de seus serviços.
>     *   Destaque os benefícios de ter flexibilidade e autonomia no trabalho.
> 10. **Conclusão:**
>     *   Reforce a ideia de que os serviços freelance são uma ótima opção para quem busca renda extra em 2025.
>     *   Incentive os leitores a explorarem as plataformas e começarem a oferecer suas habilidades.
>     *   Deixe uma mensagem inspiradora sobre a possibilidade de transformar suas paixões em fonte de renda.
> 
> Este plano fornecerá um post informativo e inspirador, incentivando os leitores a explorarem o mundo dos serviços freelance como uma forma flexível e acessível de gerar renda extra.
> 


--------------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> 🚀 Quer uma grana extra com a flexibilidade que você sempre sonhou? Em 2025, os serviços freelance estão BOMBANDO! 💥
> 
> 💻 De design gráfico a programação, passando por redação e marketing digital, a demanda por freelancers está altíssima. E o melhor? Você faz seus horários e escolhe os projetos que mais te interessam. 😉
> 
> 💰 Plataformas como Workana, Fiverr e Upwork estão cheias de oportunidades esperando por você. Quer saber como se destacar?
> 
> ✨ Dica de ouro: crie um perfil TOP, mostre suas habilidades e comece com projetos menores pra construir sua reputação. E não esqueça de divulgar seu trabalho nas redes sociais! 😉
> 
> 👉 Tá esperando o quê? Transforme suas habilidades em renda extra e conquiste a liberdade que só o trabalho freelance pode te dar! 🚀
> 
> #RendaExtra #Freelancer #Alura #TrabalhoRemoto
> 


--------------------------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais engajador e otimizado para o público do Instagram. Aqui estão algumas sugestões:
> 
> *   **Emojis:** Usar emojis é ótimo, mas podemos variar um pouco para não sobrecarregar o texto.
> *   **Chamada para Ação (CTA):** Podemos adicionar uma pergunta direta para incentivar o engajamento nos comentários.
> *   **Hashtags:** Adicionar hashtags mais específicas pode aumentar o alcance do post.
> 
> **Sugestão de versão revisada:**
> 
> 🚀 Quer uma grana extra com a flexibilidade que você sempre quis? ✨ Em 2024, os serviços freelance estão bombando!
> 
> 💻 De design gráfico a programação, passando por redação e marketing digital, a demanda por freelancers está altíssima. E o melhor? Você faz seus horários e escolhe os projetos que mais te interessam. 😉
> 
> 💰 Plataformas como Workana, Fiverr e Upwork estão cheias de oportunidades esperando por você. Quer saber como se destacar?
> 
> ✨ Dica de ouro: crie um perfil TOP, mostre suas habilidades e comece com projetos menores pra construir sua reputação. E não esqueça de divulgar seu trabalho nas redes sociais!
> 
> Qual habilidade você quer transformar em renda extra ainda hoje? 👇 Compartilhe aqui nos comentários!
> 
> \#RendaExtra \#Freelancer \#Alura \#TrabalhoRemoto \#Oportunidades \#HomeOffice \#DesenvolvimentoProfissional
> 


--------------------------------------------------------------------
